In [3]:
"""
BOPTEST HVAC 离散控制 - 完整实现(调试增强版)
使用A2C算法训练HVAC系统的离散控制策略
"""

import sys
import requests
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
import json
import time
from datetime import datetime

# =====================================================================
# BOPTEST 配置
# =====================================================================
BOPTEST_URL = "http://localhost"
TESTCASE_NAME = "bestest_air"  # 使用更简单的测试案例

# 离散动作空间定义
HEAT_PUMP_LEVELS = [0.0, 0.5, 1.0]      # 热泵：关/中/开
FAN_LEVELS = [0.0, 0.5, 1.0]            # 风扇：关/中/开  
PUMP_LEVELS = [0.0, 0.5, 1.0]           # 泵:关/中/开

# 总动作数 = 3 * 3 * 3 = 27
NUM_ACTIONS = len(HEAT_PUMP_LEVELS) * len(FAN_LEVELS) * len(PUMP_LEVELS)

# =====================================================================
# 工具函数
# =====================================================================

def log(message, level="INFO"):
    """带时间戳的日志输出"""
    timestamp = datetime.now().strftime("%H:%M:%S")
    print(f"[{timestamp}] {level:5s} | {message}")

def test_boptest_connection():
    """测试BOPTEST服务是否运行"""
    log("测试BOPTEST连接...")
    try:
        response = requests.get(f"{BOPTEST_URL}/testcases", timeout=5)
        if response.status_code == 200:
            testcases = response.json()
            log(f"✅ BOPTEST连接成功！", "INFO")
            log(f"可用测试案例数量: {len(testcases)}", "INFO")
            log(f"前5个测试案例: {[tc['testcaseid'] for tc in testcases[:5]]}", "INFO")
            return True
        else:
            log(f"❌ BOPTEST响应异常: {response.status_code}", "ERROR")
            return False
    except requests.exceptions.ConnectionError:
        log("❌ 无法连接到BOPTEST服务！", "ERROR")
        log("请确保已运行: docker compose up -d", "ERROR")
        return False
    except Exception as e:
        log(f"❌ 连接测试失败: {e}", "ERROR")
        return False

def action_index_to_values(action_idx):
    """将离散动作索引转换为具体控制值"""
    pump_idx = action_idx % len(PUMP_LEVELS)
    fan_idx = (action_idx // len(PUMP_LEVELS)) % len(FAN_LEVELS)
    heat_pump_idx = action_idx // (len(PUMP_LEVELS) * len(FAN_LEVELS))
    
    return {
        'heat_pump': HEAT_PUMP_LEVELS[heat_pump_idx],
        'fan': FAN_LEVELS[fan_idx],
        'pump': PUMP_LEVELS[pump_idx]
    }

# =====================================================================
# BOPTEST 环境封装
# =====================================================================

class BOPTESTEnv:
    """BOPTEST环境的Gym风格封装"""
    
    def __init__(self, url=BOPTEST_URL, testcase_name=TESTCASE_NAME, step_period=900, use_existing_test=False):
        self.url = url
        self.testcase_name = testcase_name
        self.step_period = step_period  # 15分钟 = 900秒
        self.testid = None
        self.use_existing_test = use_existing_test
        self.observation_space_dim = 8
        self.action_space_dim = NUM_ACTIONS
        log(f"环境初始化: 测试案例={testcase_name}, 步长={step_period}秒, use_existing={use_existing_test}", "INFO")
        
    def select_testcase(self, use_existing=False):
        """选择测试案例并获取 testid"""
        
        # 选项: 使用已存在的测试实例
        if use_existing:
            existing_testid = "b7a09a51-9255-4719-bb69-68ae5b55f068"
            log(f"🔄 尝试使用现有测试实例: {existing_testid}", "INFO")
            try:
                status_response = requests.get(
                    f"{self.url}/status/{existing_testid}",
                    timeout=10
                )
                if status_response.status_code == 200:
                    self.testid = existing_testid
                    log(f"✅ 成功连接到现有测试实例！", "INFO")
                    return True
            except Exception as e:
                log(f"无法连接到现有实例: {e}", "WARN")
        
        log(f"📋 正在选择测试案例: {self.testcase_name}", "INFO")
        
        # 首先检查测试案例是否存在
        try:
            check_response = requests.get(
                f"{self.url}/testcases",
                timeout=10
            )
            if check_response.status_code == 200:
                available = [tc['testcaseid'] for tc in check_response.json()]
                if self.testcase_name not in available:
                    log(f"❌ 测试案例 '{self.testcase_name}' 不存在", "ERROR")
                    log(f"可用案例: {available}", "ERROR")
                    return False
                log(f"✓ 测试案例存在", "DEBUG")
        except Exception as e:
            log(f"无法检查测试案例列表: {e}", "WARN")
        
        # 选择测试案例
        try:
            log(f"发送 POST 请求到: {self.url}/testcases/{self.testcase_name}/select", "DEBUG")
            response = requests.post(
                f"{self.url}/testcases/{self.testcase_name}/select",
                timeout=120  # 增加到120秒
            )
            
            log(f"响应状态码: {response.status_code}", "DEBUG")
            log(f"响应内容: {response.text[:200]}", "DEBUG")
            
            if response.status_code == 200:
                result = response.json()
                self.testid = result.get('testid')
                log(f"✅ 测试案例选择成功！Test ID: {self.testid}", "INFO")
                
                # 简单等待,不检查状态(因为状态API格式不一致)
                log("等待测试实例启动(30秒)...", "INFO")
                time.sleep(30)
                log("✅ 等待完成,假设测试实例已就绪", "INFO")
                
                return True
            else:
                log(f"❌ 选择测试案例失败: {response.status_code}", "ERROR")
                log(f"响应: {response.text}", "ERROR")
                return False
                
        except requests.exceptions.Timeout:
            log(f"❌ 选择测试案例超时(120秒)", "ERROR")
            log(f"这通常是因为 worker 正在加载 FMU 模型", "ERROR")
            log(f"建议:", "ERROR")
            log(f"1. 检查 worker 是否在运行: docker ps", "ERROR")
            log(f"2. 查看 worker 日志: docker compose logs worker --tail=100", "ERROR")
            log(f"3. 检查 worker 是否卡住: docker compose restart worker", "ERROR")
            return False
        except Exception as e:
            log(f"❌ 选择测试案例异常: {type(e).__name__}: {e}", "ERROR")
            import traceback
            log(f"堆栈跟踪:\n{traceback.format_exc()}", "DEBUG")
            return False
    
    def reset(self):
        """重置环境"""
        log("=" * 60, "INFO")
        log("🔄 重置环境", "INFO")
        
        if not self.testid:
            if not self.select_testcase(use_existing=self.use_existing_test):
                log("❌ 无法选择测试案例，环境重置失败", "ERROR")
                raise Exception("无法选择测试案例")
        
        # 初始化BOPTEST
        try:
            log(f"初始化测试: testid={self.testid}", "DEBUG")
            response = requests.put(
                f"{self.url}/initialize/{self.testid}",
                json={"start_time": 0, "warmup_period": 0},
                timeout=30
            )
            result = response.json()
            
            if result.get('status') == 200:
                log("✓ 测试初始化成功", "INFO")
            else:
                log(f"⚠️  初始化返回: {result.get('message')}", "WARN")
        except Exception as e:
            log(f"❌ 初始化失败: {e}", "ERROR")
            raise
        
        # 设置仿真步长
        try:
            log(f"设置仿真步长: {self.step_period}秒", "DEBUG")
            response = requests.put(
                f"{self.url}/step/{self.testid}",
                json={"step": self.step_period},
                timeout=10
            )
            if response.status_code == 200:
                log("✓ 步长设置成功", "INFO")
        except Exception as e:
            log(f"⚠️  设置步长警告: {e}", "WARN")
        
        # 获取初始观测
        obs = self._get_observations()
        log(f"初始观测: temp={obs[0]:.1f}°C, outdoor_temp={obs[1]:.1f}°C", "INFO")
        log("=" * 60, "INFO")
        
        return obs
    
    def step(self, action_idx):
        """执行一步仿真"""
        # 将动作索引转换为控制值
        action_values = action_index_to_values(action_idx)
        
        log(f"执行动作 #{action_idx}: HP={action_values['heat_pump']:.1f}, "
            f"Fan={action_values['fan']:.1f}, Pump={action_values['pump']:.1f}", "DEBUG")
        
        # 构造BOPTEST动作字典
        boptest_action = {
            'oveHeaPumY_u': action_values['heat_pump'],
            'oveHeaPumY_activate': 1,
            'oveFan_u': action_values['fan'],
            'oveFan_activate': 1,
            'ovePum_u': action_values['pump'],
            'ovePum_activate': 1
        }
        
        # 发送动作并推进仿真
        try:
            response = requests.post(
                f"{self.url}/advance/{self.testid}",
                json=boptest_action,
                timeout=30
            )
            result = response.json()
            
            if result.get('status') != 200:
                log(f"⚠️  推进仿真警告: {result.get('message')}", "WARN")
            
            payload = result.get('payload', {})
            sim_time = payload.get('time', 0)
            log(f"仿真时间: {sim_time:.0f}秒 ({sim_time/3600:.1f}小时)", "DEBUG")
            
        except Exception as e:
            log(f"❌ 推进仿真失败: {e}", "ERROR")
            raise
        
        # 获取新观测
        obs = self._get_observations()
        
        # 计算奖励
        reward = self._calculate_reward(payload)
        
        # 检查是否结束
        done = payload.get('time', 0) >= 86400 * 7  # 仿真7天
        
        log(f"奖励={reward:.2f}, 室温={obs[0]:.1f}°C, done={done}", "DEBUG")
        
        return obs, reward, done, payload
    
    def _get_observations(self):
        """获取当前观测"""
        try:
            response = requests.get(f"{self.url}/measurements/{self.testid}", timeout=10)
            result = response.json()
            measurements = result.get('payload', {})
            
            obs = np.array([
                measurements.get('reaTZon_y', 20.0),           # 房间温度
                measurements.get('weaSta_reaWeaTDryBul_y', 10.0),  # 室外温度
                measurements.get('weaSta_reaWeaRelHum_y', 50.0),   # 室外湿度
                measurements.get('reaQHea_y', 0.0) / 10000,    # 加热功率(归一化)
                measurements.get('reaPFan_y', 0.0) / 1000,     # 风扇功率(归一化)
                measurements.get('reaPPum_y', 0.0) / 1000,     # 泵功率(归一化)
                measurements.get('reaTSetHea_y', 20.0),        # 加热setpoint
                measurements.get('reaTSetCoo_y', 24.0)         # 冷却setpoint
            ])
            
            return obs
        except Exception as e:
            log(f"⚠️  获取观测失败: {e}", "WARN")
            return np.zeros(8)
    
    def _calculate_reward(self, payload):
        """计算奖励函数"""
        # 温度舒适度
        temp = payload.get('reaTZon_y', 20.0)
        temp_target = 21.0
        temp_penalty = -abs(temp - temp_target) * 2.0
        
        # 能耗惩罚
        power_heat = payload.get('reaQHea_y', 0.0) / 10000
        power_fan = payload.get('reaPFan_y', 0.0) / 1000
        power_pump = payload.get('reaPPum_y', 0.0) / 1000
        energy_penalty = -(power_heat + power_fan + power_pump) * 0.1
        
        # 总奖励
        reward = temp_penalty + energy_penalty + 1.0  # 基础存活奖励
        
        log(f"奖励组成: temp_penalty={temp_penalty:.2f}, "
            f"energy_penalty={energy_penalty:.2f}, total={reward:.2f}", "DEBUG")
        
        return reward
    
    def stop(self):
        """停止测试"""
        if self.testid:
            try:
                requests.put(f"{self.url}/stop/{self.testid}", timeout=10)
                log(f"✅ 测试已停止: {self.testid}", "INFO")
                self.testid = None
            except Exception as e:
                log(f"⚠️  停止测试失败: {e}", "WARN")

# =====================================================================
# A2C 算法实现
# =====================================================================

class ActorCritic(nn.Module):
    """Actor-Critic网络"""
    
    def __init__(self, state_dim, action_dim, hidden_dim=128):
        super().__init__()
        
        # 共享特征提取层
        self.shared = nn.Sequential(
            nn.Linear(state_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )
        
        # Actor头：输出动作概率
        self.actor = nn.Linear(hidden_dim, action_dim)
        
        # Critic头：输出状态价值
        self.critic = nn.Linear(hidden_dim, 1)
        
    def forward(self, state):
        features = self.shared(state)
        action_probs = torch.softmax(self.actor(features), dim=-1)
        state_value = self.critic(features)
        return action_probs, state_value

class A2CAgent:
    """A2C智能体"""
    
    def __init__(self, state_dim, action_dim, lr=0.001, gamma=0.99):
        self.device = torch.device("cpu")
        self.model = ActorCritic(state_dim, action_dim).to(self.device)
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)
        self.gamma = gamma
        log(f"A2C Agent 初始化: state_dim={state_dim}, action_dim={action_dim}, lr={lr}, gamma={gamma}", "INFO")
        
    def select_action(self, state):
        """选择动作"""
        state_tensor = torch.FloatTensor(state).unsqueeze(0).to(self.device)
        
        with torch.no_grad():
            action_probs, state_value = self.model(state_tensor)
        
        # 从概率分布中采样
        action = torch.multinomial(action_probs, 1).item()
        
        log(f"动作选择: action={action}, value={state_value.item():.3f}, "
            f"top3_probs={action_probs[0].topk(3).values.tolist()}", "DEBUG")
        
        return action
    
    def update(self, states, actions, rewards, next_states, dones):
        """更新模型"""
        # 转换为张量
        states = torch.FloatTensor(states).to(self.device)
        actions = torch.LongTensor(actions).to(self.device)
        rewards = torch.FloatTensor(rewards).to(self.device)
        next_states = torch.FloatTensor(next_states).to(self.device)
        dones = torch.FloatTensor(dones).to(self.device)
        
        # 前向传播
        action_probs, values = self.model(states)
        _, next_values = self.model(next_states)
        
        # 计算TD目标和优势
        td_target = rewards + self.gamma * next_values.squeeze() * (1 - dones)
        advantages = td_target - values.squeeze()
        
        # Actor损失（策略梯度）
        log_probs = torch.log(action_probs.gather(1, actions.unsqueeze(1)).squeeze())
        actor_loss = -(log_probs * advantages.detach()).mean()
        
        # Critic损失（TD误差）
        critic_loss = advantages.pow(2).mean()
        
        # 总损失
        loss = actor_loss + 0.5 * critic_loss
        
        # 反向传播
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
        log(f"模型更新: loss={loss.item():.4f}, actor_loss={actor_loss.item():.4f}, "
            f"critic_loss={critic_loss.item():.4f}, avg_advantage={advantages.mean().item():.3f}", "DEBUG")
        
        return loss.item()
    
    def save(self, path):
        """保存模型"""
        torch.save(self.model.state_dict(), path)
        log(f"💾 模型已保存到: {path}", "INFO")
    
    def load(self, path):
        """加载模型"""
        self.model.load_state_dict(torch.load(path))
        log(f"📂 模型已加载: {path}", "INFO")

# =====================================================================
# 训练主程序
# =====================================================================

def train(num_episodes=10, batch_size=32, max_steps_per_episode=50, use_existing_test=False):
    """训练A2C智能体"""
    
    # 创建环境和智能体
    env = BOPTESTEnv(use_existing_test=use_existing_test)
    agent = A2CAgent(state_dim=8, action_dim=NUM_ACTIONS)
    
    # 经验回放缓冲区
    buffer = {
        'states': deque(maxlen=10000),
        'actions': deque(maxlen=10000),
        'rewards': deque(maxlen=10000),
        'next_states': deque(maxlen=10000),
        'dones': deque(maxlen=10000)
    }
    
    print("\n" + "="*60)
    log("🚀 开始训练 A2C 智能体", "INFO")
    print("="*60 + "\n")
    
    for episode in range(num_episodes):
        log(f"" + "="*60, "INFO")
        log(f"📊 Episode {episode + 1}/{num_episodes}", "INFO")
        log(f"" + "="*60, "INFO")
        
        try:
            state = env.reset()
        except Exception as e:
            log(f"❌ Episode {episode + 1} 重置失败: {e}", "ERROR")
            log("尝试重新选择测试案例...", "WARN")
            env.testid = None
            time.sleep(5)
            continue
        
        episode_reward = 0
        episode_steps = 0
        episode_losses = []
        
        for step in range(max_steps_per_episode):
            log(f"\n--- Step {step + 1}/{max_steps_per_episode} ---", "INFO")
            
            # 选择动作
            action = agent.select_action(state)
            
            # 执行动作
            try:
                next_state, reward, done, info = env.step(action)
            except Exception as e:
                log(f"❌ 执行动作失败: {e}", "ERROR")
                break
            
            # 存储经验
            buffer['states'].append(state)
            buffer['actions'].append(action)
            buffer['rewards'].append(reward)
            buffer['next_states'].append(next_state)
            buffer['dones'].append(float(done))
            
            episode_reward += reward
            episode_steps += 1
            
            log(f"步骤总结: reward={reward:.2f}, 累计reward={episode_reward:.2f}", "INFO")
            
            # 每32步更新一次模型
            if len(buffer['states']) >= batch_size and episode_steps % 32 == 0:
                log("执行模型更新...", "INFO")
                # 采样一批经验
                indices = np.random.choice(len(buffer['states']), batch_size, replace=False)
                
                batch_states = [buffer['states'][i] for i in indices]
                batch_actions = [buffer['actions'][i] for i in indices]
                batch_rewards = [buffer['rewards'][i] for i in indices]
                batch_next_states = [buffer['next_states'][i] for i in indices]
                batch_dones = [buffer['dones'][i] for i in indices]
                
                loss = agent.update(batch_states, batch_actions, batch_rewards, 
                                  batch_next_states, batch_dones)
                episode_losses.append(loss)
                
                log(f"✓ 模型已更新 | Loss={loss:.4f}", "INFO")
            
            state = next_state
            
            if done:
                log("Episode 结束条件满足", "INFO")
                break
        
        # Episode 总结
        avg_loss = np.mean(episode_losses) if episode_losses else 0
        log(f"" + "="*60, "INFO")
        log(f"✅ Episode {episode + 1} 完成", "INFO")
        log(f"   总奖励: {episode_reward:.2f}", "INFO")
        log(f"   步数: {episode_steps}", "INFO")
        log(f"   平均Loss: {avg_loss:.4f}", "INFO")
        log(f"   缓冲区大小: {len(buffer['states'])}", "INFO")
        log(f"" + "="*60, "INFO")
        
        # 每5个episode保存一次模型
        if (episode + 1) % 5 == 0:
            agent.save(f"boptest_a2c_ep{episode+1}.pth")
        
        # 停止当前测试以释放资源
        env.stop()
        time.sleep(2)
    
    # 保存最终模型
    agent.save("boptest_a2c_final.pth")
    print("\n" + "="*60)
    log("🎉 训练完成！", "INFO")
    print("="*60 + "\n")

# =====================================================================
# 主函数
# =====================================================================

def main():
    """主函数"""
    
    # 检查命令行参数
    if len(sys.argv) > 1 and sys.argv[1] == "test":
        print("\n🔍 测试BOPTEST连接...")
        test_boptest_connection()
        return
    
    # 测试连接
    log("🔍 检查BOPTEST连接...", "INFO")
    if not test_boptest_connection():
        log("❌ 请先启动BOPTEST服务，然后再运行此程序。", "ERROR")
        log("启动命令:", "INFO")
        log("cd ~/Desktop/BOPTEST_Project/project1-boptest", "INFO")
        log("docker compose up -d", "INFO")
        return
    
    # 开始训练
    log("⏳ 准备开始训练...", "INFO")
    log("💡 提示: 如果选择测试案例一直超时,可以使用 use_existing_test=True 参数", "INFO")
    time.sleep(2)
    
    # 选择训练模式
    if len(sys.argv) > 2 and sys.argv[2] == "existing":
        log("使用现有测试实例模式", "INFO")
        train(num_episodes=3, max_steps_per_episode=20, use_existing_test=True)
    else:
        train(num_episodes=3, max_steps_per_episode=20, use_existing_test=False)

if __name__ == "__main__":
    main()

[15:47:12] INFO  | 🔍 检查BOPTEST连接...
[15:47:12] INFO  | 测试BOPTEST连接...
[15:47:12] INFO  | ✅ BOPTEST连接成功！
[15:47:12] INFO  | 可用测试案例数量: 12
[15:47:12] INFO  | 前5个测试案例: ['bestest_air', 'bestest_hydronic', 'bestest_hydronic_heat_pump', 'multizone_office_complex_air', 'multizone_office_simple_air']
[15:47:12] INFO  | ⏳ 准备开始训练...
[15:47:12] INFO  | 💡 提示: 如果选择测试案例一直超时,可以使用 use_existing_test=True 参数
[15:47:14] INFO  | 环境初始化: 测试案例=bestest_air, 步长=900秒, use_existing=False
[15:47:14] INFO  | A2C Agent 初始化: state_dim=8, action_dim=27, lr=0.001, gamma=0.99

[15:47:14] INFO  | 🚀 开始训练 A2C 智能体

[15:47:14] INFO  | ============================================================
[15:47:14] INFO  | 📊 Episode 1/3
[15:47:14] INFO  | ============================================================
[15:47:14] INFO  | ============================================================
[15:47:14] INFO  | 🔄 重置环境
[15:47:14] INFO  | 📋 正在选择测试案例: bestest_air
[15:47:14] DEBUG | ✓ 测试案例存在
[15:47:14] DEBUG | 发送 POST 请求到: http://localho

In [5]:
"""
BOPTEST 调试脚本 - 查询测试案例的输入/输出变量
"""

import requests
import json
import time

BOPTEST_URL = "http://localhost"
TESTCASE_NAME = "bestest_air"

def pretty_print(data, title=""):
    """美化打印JSON数据"""
    print(f"\n{'='*60}")
    print(f"📋 {title}")
    print('='*60)
    if isinstance(data, dict):
        for key, value in sorted(data.items()):
            print(f"  {key}: {value}")
    elif isinstance(data, list):
        for item in data:
            print(f"  - {item}")
    else:
        print(json.dumps(data, indent=2))

def main():
    print("🔍 BOPTEST 调试工具")
    print("="*60)
    
    # 1. 检查连接
    print("\n1️⃣ 检查BOPTEST连接...")
    try:
        resp = requests.get(f"{BOPTEST_URL}/testcases", timeout=5)
        testcases = resp.json()
        print(f"✅ 连接成功，共 {len(testcases)} 个测试案例")
    except Exception as e:
        print(f"❌ 连接失败: {e}")
        return
    
    # 2. 选择测试案例
    print(f"\n2️⃣ 选择测试案例: {TESTCASE_NAME}")
    try:
        resp = requests.post(
            f"{BOPTEST_URL}/testcases/{TESTCASE_NAME}/select",
            timeout=60
        )
        result = resp.json()
        testid = result.get('testid')
        print(f"✅ 测试案例已选择，testid: {testid}")
    except Exception as e:
        print(f"❌ 选择失败: {e}")
        return
    
    # 等待实例启动
    print("\n⏳ 等待测试实例启动 (10秒)...")
    time.sleep(10)
    
    # 3. 查询可用输入变量 (控制信号)
    print(f"\n3️⃣ 查询可用输入变量 (控制信号)...")
    try:
        resp = requests.get(f"{BOPTEST_URL}/inputs/{testid}", timeout=10)
        inputs = resp.json()
        
        if 'payload' in inputs:
            inputs = inputs['payload']
        
        pretty_print(inputs, "可用输入变量 (INPUTS)")
        
        # 特别标注 _activate 和 _u 变量
        print("\n📌 控制变量分析:")
        control_vars = {}
        for key in inputs.keys():
            if key.endswith('_u'):
                base_name = key[:-2]
                if base_name not in control_vars:
                    control_vars[base_name] = {}
                control_vars[base_name]['value'] = key
            elif key.endswith('_activate'):
                base_name = key[:-9]
                if base_name not in control_vars:
                    control_vars[base_name] = {}
                control_vars[base_name]['activate'] = key
        
        print("\n可用的控制器:")
        for name, vars in control_vars.items():
            print(f"  🎛️  {name}:")
            print(f"      值变量: {vars.get('value', 'N/A')}")
            print(f"      激活变量: {vars.get('activate', 'N/A')}")
            if name in inputs:
                info = inputs.get(vars.get('value', ''), {})
                print(f"      范围: {info}")
                
    except Exception as e:
        print(f"❌ 查询输入失败: {e}")
        import traceback
        traceback.print_exc()
    
    # 4. 查询可用测量变量 (观测)
    print(f"\n4️⃣ 查询可用测量变量 (观测)...")
    try:
        resp = requests.get(f"{BOPTEST_URL}/measurements/{testid}", timeout=10)
        measurements = resp.json()
        
        if 'payload' in measurements:
            measurements = measurements['payload']
        
        pretty_print(measurements, "可用测量变量 (MEASUREMENTS)")
        
    except Exception as e:
        print(f"❌ 查询测量失败: {e}")
    
    # 5. 查询前进预测变量
    print(f"\n5️⃣ 查询预测变量...")
    try:
        resp = requests.get(f"{BOPTEST_URL}/forecast_points/{testid}", timeout=10)
        forecast = resp.json()
        
        if 'payload' in forecast:
            forecast = forecast['payload']
        
        print(f"预测变量数量: {len(forecast) if isinstance(forecast, (list, dict)) else 'N/A'}")
        if isinstance(forecast, dict):
            print("前10个预测变量:")
            for i, key in enumerate(list(forecast.keys())[:10]):
                print(f"  - {key}")
                
    except Exception as e:
        print(f"⚠️ 查询预测失败: {e}")
    
    # 6. 停止测试
    print(f"\n6️⃣ 停止测试实例...")
    try:
        requests.put(f"{BOPTEST_URL}/stop/{testid}", timeout=10)
        print("✅ 测试已停止")
    except:
        pass
    
    print("\n" + "="*60)
    print("🎯 调试完成！请根据上面的输入变量修改你的代码")
    print("="*60)

if __name__ == "__main__":
    main()

🔍 BOPTEST 调试工具

1️⃣ 检查BOPTEST连接...
✅ 连接成功，共 12 个测试案例

2️⃣ 选择测试案例: bestest_air
✅ 测试案例已选择，testid: db60e8f1-f753-4947-80d8-af1ffc7b5e8a

⏳ 等待测试实例启动 (10秒)...

3️⃣ 查询可用输入变量 (控制信号)...

📋 可用输入变量 (INPUTS)
  con_oveTSetCoo_activate: {'Unit': None, 'Description': 'Activation for Zone temperature setpoint for cooling', 'Minimum': None, 'Maximum': None}
  con_oveTSetCoo_u: {'Unit': 'K', 'Description': 'Zone temperature setpoint for cooling', 'Minimum': 278.15, 'Maximum': 308.15}
  con_oveTSetHea_activate: {'Unit': None, 'Description': 'Activation for Zone temperature setpoint for heating', 'Minimum': None, 'Maximum': None}
  con_oveTSetHea_u: {'Unit': 'K', 'Description': 'Zone temperature setpoint for heating', 'Minimum': 278.15, 'Maximum': 308.15}
  fcu_oveFan_activate: {'Unit': None, 'Description': 'Activation for Fan control signal as air mass flow rate normalized to the design air mass flow rate', 'Minimum': None, 'Maximum': None}
  fcu_oveFan_u: {'Unit': '1', 'Description': 'Fan control signa